# [Create Agent with Azure Functions](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-function-call)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

The **STORAGE_SERVICE_ENDPOINT** string is used to triggering the Azure function.<br/>

Notes:
- Inspired by [sample_agents_azure_functions.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_azure_functions.py)
- [Getting Started with Azure Functions](https://learn.microsoft.com/azure/azure-functions/functions-get-started) page for more information on Azure Functions
- [Azure Blob storage bindings for Azure Functions overview](https://learn.microsoft.com/en-us/azure/azure-functions/functions-bindings-storage-blob?tabs=isolated-process%2Cextensionv5%2Cextensionv3&pivots=programming-language-python)

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"
project_conn_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_conn_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureFunctionTool, AzureFunctionStorageQueue # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE FUNCTIONS
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_conn_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [3]:
storage_service_endpoint = os.environ["STORAGE_SERVICE_ENDPOINT"]

In [4]:
azure_function_tool = AzureFunctionTool(
    name="foo",
    description="Get answers from the foo bot.",
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "The question to ask."},
            "outputqueueuri": {"type": "string", "description": "The full output queue uri."},
        },
    },
    input_queue=AzureFunctionStorageQueue(
        queue_name="azure-function-foo-input",
        storage_service_endpoint=storage_service_endpoint,
    ),
    output_queue=AzureFunctionStorageQueue(
        queue_name="azure-function-tool-output",
        storage_service_endpoint=storage_service_endpoint,
    ),
)

print(f"azure_function_tool.definitions: {azure_function_tool.definitions}")
print(f"\nazure_function_tool.resources: {azure_function_tool.resources}")

azure_function_tool.definitions: [{'type': 'azure_function', 'azure_function': {'function': {'name': 'foo', 'description': 'Get answers from the foo bot.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The question to ask.'}, 'outputqueueuri': {'type': 'string', 'description': 'The full output queue uri.'}}}}, 'input_binding': {'storage_queue': {'queue_name': 'azure-function-foo-input', 'queue_service_endpoint': 'foo'}, 'type': 'storage_queue'}, 'output_binding': {'storage_queue': {'queue_name': 'azure-function-tool-output', 'queue_service_endpoint': 'foo'}, 'type': 'storage_queue'}}}]

azure_function_tool.resources: {}


# Just for testing: invoke the Azure Function

In [5]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="azure-function-agent-foo",
    instructions="""
    You are a helpful support agent. Use the provided function any time the prompt contains the string 'What would foo say?'. 
    When you invoke the function, ALWAYS specify the output queue uri parameter as '{storage_service_endpoint}/azure-function-tool-output'. 
    Always responds with \"Foo says\" and then the response from the tool.
    """,
    tools=azure_function_tool.definitions,
    tool_resources=azure_function_tool.resources,
)

print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_2xW3jmeofGBUdS2rPTqhKc3J


# Create the thread and attach a new message to it

In [6]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What is the most prevalent element in the universe? What would foo say?" #  What would foo say?
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'object': 'thread', 'created_at': 1737219700, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_YYe4DFq6FK7ApnNstfrXG5KO', 'object': 'thread.message', 'created_at': 1737219700, 'assistant_id': None, 'thread_id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is the most prevalent element in the universe?', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously

In [7]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_03Z6oEm3Uog9DBHhwpiBPRWo', 'object': 'thread.run', 'created_at': 1737219701, 'assistant_id': 'asst_2xW3jmeofGBUdS2rPTqhKc3J', 'thread_id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'status': 'completed', 'started_at': 1737219701, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1737219702, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': '\n    You are a helpful support agent. Use the provided function any time the prompt contains the string \'What would foo say?\'. \n    When you invoke the function, ALWAYS specify the output queue uri parameter as \'{storage_service_endpoint}/azure-function-tool-output\'. \n    Always responds with "Foo says" and then the response from the tool.\n    ', 'tools': [{'type': 'azure_function', 'azure_function': {'input_binding': {'type': 'storage_queue', 'storage_queue': {'queue_service_endpoint': 'foo', 'queue_name': 'azure-function-f

# Fetch messages from the thread after the agent run execution

In [8]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Here are the {len(messages.data)} messages, starting with the most recent one:\n")
    i=0
    for m in messages.data:
        j = 0
        i += 1
        print(f"\n===== MESSAGE {i} =====")
        for c in m.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages, starting with the most recent one:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: The most prevalent element in the universe is hydrogen. It makes up about 75% of the normal matter in the universe by mass and over 90% of the atoms in the universe.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: What is the most prevalent element in the universe?


# Run Steps

In [9]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 1

Run step 1: {'id': 'step_UxEy2HLsftqREAwEwQ0tLdIO', 'object': 'thread.run.step', 'created_at': 1737219701, 'run_id': 'run_03Z6oEm3Uog9DBHhwpiBPRWo', 'assistant_id': 'asst_2xW3jmeofGBUdS2rPTqhKc3J', 'thread_id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1737219702, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_mbWCzhlV2aJbfAZFv45kDxaH'}}, 'usage': {'prompt_tokens': 153, 'completion_tokens': 40, 'total_tokens': 193}} 



# START teardown

In [10]:
print(f"Deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

Deleting trhead: {'id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'object': 'thread', 'created_at': 1737219700, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_lINRKTD9vlDoCNGqx41USgAH', 'object': 'thread.deleted', 'deleted': True}

In [11]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

2 agent(s) will now be deleted

1 - Agent azure-function-agent-foo has been deleted

2 - Agent azure-function-agent-foo has been deleted


# HIC SUNT LEONES